## Machine Learning Pipeline for Income Prediction

This script implements a machine learning pipeline to predict income levels based on various features.

The pipeline includes data preprocessing, feature transformation, model training, and evaluation.

In [ ]:
from src.utils.utils import load_data, save_model,

In [ ]:
if __name__ == "__main__":
    # Pipeline principal
    main_pipeline = train_pipeline()
    
    # Casos especiales
    print("\nTraining Non-Gaussian Pipeline:")
    poisson_model = non_gaussian_pipeline()
    
    print("\nTraining Imbalanced Classification Pipeline:")
    imbalanced_model = imbalanced_classification_pipeline()

### Training the model